### Programa para obtener datos de idealista  
El programa te permite elegir comunidad autónoma, provincia y localidad.  
Después te solicita el radio de búsqueda.  
Finalmente te vuelca la consulta en un fichero xlsx.

In [2]:
import pandas as pd
import json
import urllib
import requests as rq
import base64

In [3]:
import ipywidgets as widgets
from IPython.display import display

In [4]:
def obtener_coordenadas(df,localidad):
    df_filtrado = df[df['Población'] == localidad]
    latitud = df_filtrado['Latitud'].values[0]
    longitud = df_filtrado['Longitud'].values[0]
    return latitud, longitud  

In [6]:
def get_oauth_token(): 

    url = "https://api.idealista.com/oauth/token"    

    apikey= 'vlokpni9yqgvkjkrygzpfpsf6skfdt49' #sent by idealista
    secret= 'hyiDSleWFr29'  #sent by idealista
    apikey_secret = apikey + ':' + secret
    auth = str(base64.b64encode(bytes(apikey_secret, 'utf-8')))[2:][:-1]

    headers = {'Authorization' : 'Basic ' + auth,'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'}
    params = urllib.parse.urlencode({'grant_type':'client_credentials'}) #,'scope':'read'
    content = rq.post(url,headers = headers, params=params)
    bearer_token = json.loads(content.text)['access_token']

    return bearer_token

In [7]:
def search_api(token, URL):  
    headers = {'Content-Type': 'Content-Type: multipart/form-data;', 'Authorization' : 'Bearer ' + token}
    content = rq.post(url, headers = headers)
    result = json.loads(content.text)
    return result

In [8]:
# Función para actualizar las opciones de la lista desplegable de la Provincia
def update_provincias(*args):
    df_filtrado = df[df['Comunidad'] == comunidad_dropdown.value]
    provincias = df_filtrado['Provincia'].unique()
    provincia_dropdown.options = provincias

In [9]:
# Función para actualizar las opciones de la lista desplegable de la Población
def update_poblaciones(*args):
    df_filtrado = df[(df['Comunidad'] == comunidad_dropdown.value) & (df['Provincia'] == provincia_dropdown.value)]
    poblaciones = df_filtrado['Población'].unique()
    poblacion_dropdown.options = poblaciones

In [10]:
def crear_slider_radio():
    # Crea un slider para seleccionar los kilómetros de radio
    radio_slider = widgets.IntSlider(
        value=10,  # valor inicial
        min=5,  # valor mínimo
        max=50,  # valor máximo
        step=5,  # incremento
        description='Radio (km):',
        continuous_update=False
    )
    return radio_slider

In [11]:
df = pd.read_excel("./data/municipios_esp.xlsx", engine = "openpyxl")

In [12]:
# Crea una lista desplegable para la Comunidad Autónoma
comunidades = df['Comunidad'].unique()
comunidad_dropdown = widgets.Dropdown(options=comunidades)
display(comunidad_dropdown)

# Crea una lista desplegable para la Provincia
provincia_dropdown = widgets.Dropdown(options=[])
display(provincia_dropdown)

# Crea una lista desplegable para la Población
poblacion_dropdown = widgets.Dropdown(options=[])
display(poblacion_dropdown)

comunidad_dropdown.observe(update_provincias, 'value')
provincia_dropdown.observe(update_poblaciones, 'value')

# Llama a las funciones para inicializar las listas desplegables
update_provincias()
update_poblaciones()

Dropdown(options=('Andalucía', 'Aragón', 'Asturias', 'Canarias', 'Cantabria', 'Castilla La Mancha', 'Castilla …

Dropdown(options=(), value=None)

Dropdown(options=(), value=None)

In [13]:
# Crea y muestra el slider
slider = crear_slider_radio()
display(slider)

IntSlider(value=10, continuous_update=False, description='Radio (km):', max=50, min=5, step=5)

In [14]:
latitud, longitud = obtener_coordenadas(df, poblacion_dropdown.value)

In [15]:
country = 'es' #values: es, it, pt
locale = 'es' #values: es, it, pt, en, ca
language = 'es' #
max_items = '50'
operation = 'sale' 
property_type = 'homes'
order = 'priceDown' 
center = f'{latitud},{longitud}'
distance = f'{slider.value*1000}'
sort = 'desc'
bankOffer = 'false'

In [16]:
country = 'es' #values: es, it, pt
locale = 'es' #values: es, it, pt, en, ca
language = 'es' #
max_items = '50'
operation = 'rent' 
property_type = 'homes'
order = 'priceDown' 
center = f'{latitud},{longitud}'
distance = f'{slider.value*1000}'
sort = 'desc'
bankOffer = 'false'

In [16]:
print(f'{center}-{poblacion_dropdown.value}-radio:{distance} metros')

37.44359,-4.273436-Carcabuey-radio:20000 metros


In [17]:
df_tot = pd.DataFrame()
limit = 10

In [18]:
for i in range(1,limit):
    url = ('https://api.idealista.com/3.5/'+country+'/search?operation='+operation+#"&locale="+locale+
           '&maxItems='+max_items+
           '&order='+order+
           '&center='+center+
           '&distance='+distance+
           '&propertyType='+property_type+
           '&sort='+sort+ 
           '&numPage=%s'+
           '&language='+language) %(i)  
    a = search_api(get_oauth_token(), url)
    df = pd.DataFrame.from_dict(a['elementList'])
    df_tot = pd.concat([df_tot,df])

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
df_tot = df_tot.reset_index()

In [ ]:
df_tot.to_excel(f"./output/{poblacion_dropdown.value}_{slider.value}.xlsx", index = False)

In [ ]:
df_tot.to_excel(f"./output/rent_{poblacion_dropdown.value}_{slider.value}.xlsx", index = False)